# Plotting Band Structures

There are a couple of different ways of plotting band Structures.

1. p4v
2. Pedram's script
3. PyProcar

# 1. p4v

Open p4v and load vasp.xml. 

Also, export as bands.dat or bands.ags. Then run with xmgrace.`

In [ ]:
xmgrace 'bands.ags'

# 2. Pedram's script

To read DOS

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import argparse
from matplotlib.ticker import AutoMinorLocator
parser = argparse.ArgumentParser()
parser.add_argument('-legend',nargs='+',help='legen list')
parser.add_argument('-xlim',nargs=2,type=float,help='x limits')
parser.add_argument('-ylim',nargs=2,type=float,help='y limits')
parser.add_argument('-input',dest='f',help='file name')
parser.add_argument('-title',help='plot title')
parser.add_argument('-orbitals',nargs='+',type=int,help='orbitals to be ploted')
args = parser.parse_args()
norb = len(args.orbitals)
data = np.loadtxt(args.f)
ndata = 3000
from matplotlib import rc
rc('text', usetex=True)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'serif'
fig , ax = plt.subplots()
minorLocator = AutoMinorLocator()
ax.xaxis.set_minor_locator(minorLocator)
plt.tick_params(axis='x',direction='in',length=7,width=2,which='major')
plt.tick_params(axis='x',direction='in',length=4,width=2,which='minor')
plt.tick_params(axis='x',top='on')
plt.tick_params(axis='x',top='on',direction='in',length=4,width=2,which='minor')
plt.tick_params(axis='y',labelleft=0,left=0)
if args.orbitals == None :
    orbitals = range(1,len(data)/ndata)
else :
    orbitals = args.orbitals
if args.legend == None :
    legend = range(len(data)/ndata)
else :
    legend = args.legend
for iorb in orbitals:
    plt.plot(data[iorb*ndata:(iorb+1)*ndata,0],data[iorb*ndata:(iorb+1)*ndata,1],label = legend[iorb-1],linewidth=2)
    print(len(data[iorb*ndata:(iorb+1)*ndata,0]))
if args.xlim != None :
    plt.xlim(args.xlim[0],args.xlim[1])
if args.ylim != None :
    plt.ylim(args.ylim[0],args.ylim[1])
if args.title != None :
    plt.title(args.title,fontsize=12)
#plt.xticks(direction='in')
#plt.get_minor_ticks()
plt.xlabel('Energy (eV)',fontsize=16)
plt.ylabel('Density of States(a.u.)',fontsize=16)
plt.legend(fontsize=12)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
plt.axvline(x = 0.0 ,linewidth=2,color='#6309bf',linestyle='--')
plt.savefig('DOS.pdf')
plt.show()

In [ ]:
python plot_DOS.py

To plot bands

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import argparse
from matplotlib.ticker import AutoMinorLocator
import re
parser = argparse.ArgumentParser()
parser.add_argument('-legend',nargs='+',help='legen list')
parser.add_argument('-xlim',nargs=2,type=int,help='x limits')
parser.add_argument('-ylim',nargs=2,type=float,help='y limits')
parser.add_argument('-input',dest='f',help='file name')
parser.add_argument('-title',help='plot title')
from matplotlib import rc
rc('text', usetex=True)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'serif'
args = parser.parse_args()
data = np.loadtxt(args.f)
nbands = np.sum(np.sort(data) == 0)
ndata_x = len(data)/nbands
fig , ax = plt.subplots(figsize=(11,8))
for iband in range(nbands) :
    idata = data[iband*ndata_x:(iband+1)*ndata_x]
    plt.plot(idata[:,0],idata[:,1],color='b',linewidth=2)
#'#4863A0'
try :
    f = open('KPOINTS')
    lines = f.readlines()
    f.close()
    f = open('KPOINTS')
    KP = f.read()
    f.close()
except :
    print("There is no file named KPOINTS")
KP1 = re.findall('reciprocal[\s\S]*',KP)
tick_labels_raw = re.findall('!\s(.*)',KP1[0])
tick_labels_raw = np.array(tick_labels_raw)
nkpoint = int(lines[1].split()[0])

tick_pos = []
minorLocator = AutoMinorLocator()
ax.yaxis.set_minor_locator(minorLocator)
plt.tick_params(axis='y',direction='in',length=7,width=2,which='major')
plt.tick_params(axis='y',direction='in',length=4,width=2,which='minor')
plt.tick_params(axis='y',right='on',direction='in',length=7,width=2,which='major')
plt.tick_params(axis='y',right='on',direction='in',length=4,width=2,which='minor')
plt.tick_params(axis='y',top='on',direction='in',length=4,width=2,which='minor')
plt.tick_params(axis='x',bottom='off')
if args.ylim != None :
    plt.ylim(args.ylim[0],args.ylim[1])
    for i in range(0,len(idata),nkpoint):
        plt.axvline(x = idata[:,0][i],ymin=args.ylim[0],ymax=args.ylim[1],linewidth=2,color='black')
        tick_pos.append(idata[:,0][i])
    plt.axvline(x = idata[-1][0],ymin=args.ylim[0],ymax=args.ylim[1],linewidth=2,color='black')
    tick_pos.append(idata[-1][0])
else :
    tick_pos = []
    ymin, ymax = ax.get_ylim()
    plt.ylim(ymin,ymax)
    for i in range(0,len(idata),nkpoint):
        plt.axvline(x = idata[:,0][i],ymin=ymin,ymax=ymax,linewidth=2,color='black')
        tick_pos.append(idata[:,0][i])
    plt.axvline(x = idata[-1][0],ymin=ymin,ymax=ymax,linewidth=2,color='black')
    tick_pos.append(idata[-1][0])

tick_labels = np.empty(len(tick_pos),dtype='|S16')
for ilabel in range(0,len(tick_labels_raw),2):
    if ilabel == 0 :
        continue
    if tick_labels_raw[ilabel-1] == tick_labels_raw[ilabel]:
        tick_labels[ilabel/2] = tick_labels_raw[ilabel]
    else :
        tick_labels[ilabel/2] = tick_labels_raw[ilabel-1] +' '+ tick_labels_raw[ilabel]
tick_labels[0] = tick_labels_raw[0]
tick_labels[-1] = tick_labels_raw[-1]
print(tick_labels)
#for ilabel in range(len(tick_labels)):
#   tick_labels[ilabel] =  '$'+tick_labels[ilabel]+'$'
plt.xticks(tick_pos,tick_labels, fontsize=24,fontweight='bold')
plt.yticks(fontsize=20)
if args.xlim != None :
    xmin = tick_pos[args.xlim[0]]
    xmax = tick_pos[args.xlim[1]]
    plt.xlim(xmin,xmax)
else :
    plt.xlim(0,idata[-1][0])

if args.title != None :
    plt.title(args.title,fontsize=20)
plt.ylabel('Energy (eV)',fontsize=20)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(2)
plt.axhline(y = 0.0 ,linewidth=2,color='#bf2909',linestyle='--')

plt.savefig('BANDS.pdf')
plt.show()

# 3. PyPROCAR

1. Use <a href="www.aflowlib.org">aflowlib</a> to get KPOINT file for your POSCAR. 

Then run vasp. 

Repair PROCAR file

In [ ]:
procar.py repair PROCAR PROCAR_repair

Plot.

In [ ]:
procar.py bandsplot --outcar OUTCAR PROCAR_repaired -m parametric --kticks 0 15 31 47 --kname GM X W K -s 1 -o 1 2 3

--kticks would be repeated for each kpoint from the number of kpoints specified in your kpoints file. 